### Neural Machine Translation using word level language model and embeddings in Keras

In [0]:
!pip install sklearn


In [0]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.cross_validation import train_test_split
# Building a english to french translator

ModuleNotFoundError: ignored

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}4/AABWMqnEy-sS_jvqJVFESId6q-4vtM9NsQ0XT7vCoGoh9-1JZSi4RSk


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131289 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive
import sys
sys.path.append('drive/chatbot')


In [0]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [0]:
lines= pd.read_table('drive/newaConv.txt', names=['eng', 'hi'])

In [0]:
lines = lines[0:50000]

In [0]:
lines.shape

(84, 2)

In [0]:
print(lines.sample(10))

                                                    eng  \
719                                            Thank u    
345   Batana jaye tab Yup Just finished work Will se...   
170                                               Cool    
1249                                             Ha ji    
210                 kuch plan nhi h papa ka mood off h    
378   Ok Hia pe aayega Tu le aana waha se 095091 204...   
1323                                    Na gt Shopping    
249   oka phuch jaye to bata dena Your bus ticket bo...   
801                               Kewl Cool Aa ja phir    
1161       🎊🎉🎊 Wishing you a very 😊Happy Diwali....🎊🎉🎊    

                                                     hi  
719   👍🏻 ससुराल भी कीमती होता है पर मायका आखिर मायका...  
345                        Mujhe sorry kyu KR rhi ho😂😂   
170                               M khne wala tha ye😂😂   
1249                                     Ok good night   
210                                             Office   
37

#### Data Cleanup

In [0]:
lines.eng=lines.eng.apply(lambda x: str(x).lower())
lines.hi=lines.hi.apply(lambda x: str(x).lower())

In [0]:
# Take the length as 50
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
lines.hi=lines.hi.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

In [0]:
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.hi=lines.hi.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.hi=lines.hi.apply(lambda x: x.translate(remove_digits))

In [0]:
lines.head(10)

,eng,hi
0,shanky Happy 18th bday shanky :) 🎂🎂🎂🎂🍫🍫🍫🍫🍫 htt...,Mujhe holi ki photos send karo Aur group m add...
1,Bad me,Ok Ab baje do H
2,Hi chachu phn kar rhe h na,M Abhi room m nhi hu
3,Ok,Baad m karte h
4,Ok https://www.airtel.in/s/selfcare/?normalLogin,Aap ko PTA h kya ki m Ghar aa rha hu
5,Ha,Cool
6,Aja ladeshwar tere liye aarti ki thali tayar r...,🤣 Aarti ka PTA nhi Khane ki thali tayar rakhna🤣🤣
7,Charger laya h ke chore,Bus m chadne k baad yaar aaya Nhi laya
8,Bekoof Khud ka chatger laya h??,Nhi
9,Yar had h tu,Kya karu


#### Generate synthetic data

In [0]:
lines.hi = lines.hi.apply(lambda x : 'START_ '+ x + ' _END')

In [0]:
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
    
all_french_words=set()
for hi in lines.hi:
    for word in hi.split():
        if word not in all_french_words:
            all_french_words.add(word)

In [0]:
len(all_eng_words), len(all_french_words)

(1743, 1111)

In [0]:
lenght_list=[]
for l in lines.hi:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

70

In [0]:
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

63

In [0]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_french_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_french_words)
max_encoder_seq_length = max([len(input_text.split(' ')) for input_text in lines.eng])
max_decoder_seq_length = max([len(target_text.split(' ')) for target_text in lines.hi])
print(num_decoder_tokens)

# del all_eng_words, all_french_words

1111


In [0]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [0]:
len(lines.hi)*16*num_decoder_tokens

24708640

In [0]:
encoder_input_data = np.zeros(
    (len(lines.eng), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(lines.hi), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(lines.hi), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(lines.eng, lines.hi)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

### Build keras encoder-decoder model

In [0]:
embedding_size = 50

In [0]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

Using TensorFlow backend.


#### Encoder model

In [0]:
encoder_inputs = Input(shape=(None,) ,name='encoder_inputs')
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
print(encoder_inputs.name)

encoder_inputs:0


#### Decoder model

In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,),name='decoder_inputs')

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax',name='decoder_output')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
print(decoder_outputs.name)

decoder_output_1/truediv:0


In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, None)         0                                            
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     87150       encoder_inputs[0][0]             
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     55550       decoder_inputs[0][0]             
__________________________________________________________________________________________________
lstm_1 (LS

#### Fit the model

In [0]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=400,
          validation_split=0.05)

Train on 1320 samples, validate on 70 samples
Epoch 1/400
1320/1320 [==============================] - 3s 2ms/step - loss: 0.4993 - acc: 0.0143 - val_loss: 0.5173 - val_acc: 0.0143
Epoch 2/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4673 - acc: 0.0143 - val_loss: 0.4987 - val_acc: 0.0143
Epoch 3/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4533 - acc: 0.0143 - val_loss: 0.4930 - val_acc: 0.0143
Epoch 4/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4487 - acc: 0.0143 - val_loss: 0.4907 - val_acc: 0.0143
Epoch 5/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.4466 - acc: 0.0143 - val_loss: 0.4895 - val_acc: 0.0143
Epoch 6/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.4447 - acc: 0.0143 - val_loss: 0.4864 - val_acc: 0.0143
Epoch 7/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.4427 - acc: 0.0143 - val_loss: 0.4846 - val_acc: 0.0143
Epoch 8/4

1320/1320 [==============================] - 3s 3ms/step - loss: 0.4338 - acc: 0.0143 - val_loss: 0.4767 - val_acc: 0.0143
Epoch 12/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4315 - acc: 0.0143 - val_loss: 0.4753 - val_acc: 0.0143
Epoch 13/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4291 - acc: 0.0143 - val_loss: 0.4734 - val_acc: 0.0143
Epoch 14/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4268 - acc: 0.0143 - val_loss: 0.4702 - val_acc: 0.0143
Epoch 15/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4244 - acc: 0.0143 - val_loss: 0.4689 - val_acc: 0.0143
Epoch 16/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4221 - acc: 0.0143 - val_loss: 0.4665 - val_acc: 0.0143
Epoch 17/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4199 - acc: 0.0143 - val_loss: 0.4646 - val_acc: 0.0143
Epoch 18/400
1320/1320 [==============================] - 3s 

Epoch 22/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4091 - acc: 0.0143 - val_loss: 0.4563 - val_acc: 0.0143
Epoch 23/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4073 - acc: 0.0143 - val_loss: 0.4546 - val_acc: 0.0141
Epoch 24/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4055 - acc: 0.0143 - val_loss: 0.4528 - val_acc: 0.0143
Epoch 25/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4034 - acc: 0.0144 - val_loss: 0.4517 - val_acc: 0.0143
Epoch 26/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4019 - acc: 0.0144 - val_loss: 0.4504 - val_acc: 0.0143
Epoch 27/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.4000 - acc: 0.0144 - val_loss: 0.4490 - val_acc: 0.0145
Epoch 28/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.3984 - acc: 0.0144 - val_loss: 0.4472 - val_acc: 0.0143
Epoch 29/400
1320/1320 [========================

1320/1320 [==============================] - 3s 3ms/step - loss: 0.3919 - acc: 0.0147 - val_loss: 0.4425 - val_acc: 0.0149
Epoch 33/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.3905 - acc: 0.0148 - val_loss: 0.4407 - val_acc: 0.0151
Epoch 34/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.3890 - acc: 0.0148 - val_loss: 0.4390 - val_acc: 0.0151
Epoch 35/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.3873 - acc: 0.0151 - val_loss: 0.4368 - val_acc: 0.0153
Epoch 36/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.3861 - acc: 0.0150 - val_loss: 0.4368 - val_acc: 0.0151
Epoch 37/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.3844 - acc: 0.0152 - val_loss: 0.4351 - val_acc: 0.0147
Epoch 38/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.3829 - acc: 0.0153 - val_loss: 0.4333 - val_acc: 0.0151
Epoch 39/400
1320/1320 [==============================] - 4s 

Epoch 43/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.3758 - acc: 0.0156 - val_loss: 0.4287 - val_acc: 0.0149
Epoch 44/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.3742 - acc: 0.0157 - val_loss: 0.4262 - val_acc: 0.0153
Epoch 45/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.3730 - acc: 0.0158 - val_loss: 0.4247 - val_acc: 0.0155
Epoch 46/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.3714 - acc: 0.0159 - val_loss: 0.4240 - val_acc: 0.0155
Epoch 47/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.3700 - acc: 0.0160 - val_loss: 0.4221 - val_acc: 0.0155
Epoch 48/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.3685 - acc: 0.0161 - val_loss: 0.4213 - val_acc: 0.0155
Epoch 49/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.3670 - acc: 0.0162 - val_loss: 0.4195 - val_acc: 0.0161
Epoch 50/400
1320/1320 [========================

...

1320/1320 [==============================] - 4s 3ms/step - loss: 0.0762 - acc: 0.0649 - val_loss: 0.1062 - val_acc: 0.0637
Epoch 357/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0751 - acc: 0.0651 - val_loss: 0.1062 - val_acc: 0.0622
Epoch 358/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0750 - acc: 0.0652 - val_loss: 0.1115 - val_acc: 0.0633
Epoch 359/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0741 - acc: 0.0655 - val_loss: 0.1063 - val_acc: 0.0633
Epoch 360/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0742 - acc: 0.0655 - val_loss: 0.1059 - val_acc: 0.0629
Epoch 361/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0745 - acc: 0.0652 - val_loss: 0.1040 - val_acc: 0.0631
Epoch 362/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0745 - acc: 0.0651 - val_loss: 0.1046 - val_acc: 0.0633
Epoch 363/400
1320/1320 [==============================

1320/1320 [==============================] - 3s 3ms/step - loss: 0.0742 - acc: 0.0652 - val_loss: 0.1075 - val_acc: 0.0635
Epoch 367/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.0736 - acc: 0.0652 - val_loss: 0.1065 - val_acc: 0.0635
Epoch 368/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.0731 - acc: 0.0653 - val_loss: 0.1015 - val_acc: 0.0639
Epoch 369/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0717 - acc: 0.0657 - val_loss: 0.1026 - val_acc: 0.0637
Epoch 370/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0729 - acc: 0.0655 - val_loss: 0.1045 - val_acc: 0.0633
Epoch 371/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.0719 - acc: 0.0656 - val_loss: 0.1073 - val_acc: 0.0635
Epoch 372/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0753 - acc: 0.0650 - val_loss: 0.1009 - val_acc: 0.0643
Epoch 373/400
1320/1320 [==============================

1320/1320 [==============================] - 4s 3ms/step - loss: 0.0719 - acc: 0.0655 - val_loss: 0.0992 - val_acc: 0.0643
Epoch 377/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0703 - acc: 0.0660 - val_loss: 0.1049 - val_acc: 0.0637
Epoch 378/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0703 - acc: 0.0661 - val_loss: 0.0977 - val_acc: 0.0661
Epoch 379/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0716 - acc: 0.0657 - val_loss: 0.0984 - val_acc: 0.0655
Epoch 380/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0712 - acc: 0.0655 - val_loss: 0.0987 - val_acc: 0.0655
Epoch 381/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.0698 - acc: 0.0660 - val_loss: 0.1024 - val_acc: 0.0645
Epoch 382/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.0701 - acc: 0.0658 - val_loss: 0.0975 - val_acc: 0.0659
Epoch 383/400
1320/1320 [==============================

1320/1320 [==============================] - 4s 3ms/step - loss: 0.0694 - acc: 0.0661 - val_loss: 0.0945 - val_acc: 0.0661
Epoch 387/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0688 - acc: 0.0661 - val_loss: 0.0942 - val_acc: 0.0653
Epoch 388/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.0692 - acc: 0.0660 - val_loss: 0.0960 - val_acc: 0.0657
Epoch 389/400
1320/1320 [==============================] - 4s 3ms/step - loss: 0.0688 - acc: 0.0661 - val_loss: 0.0943 - val_acc: 0.0659
Epoch 390/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.0699 - acc: 0.0658 - val_loss: 0.0966 - val_acc: 0.0649
Epoch 391/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.0679 - acc: 0.0664 - val_loss: 0.1045 - val_acc: 0.0641
Epoch 392/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.0694 - acc: 0.0661 - val_loss: 0.0930 - val_acc: 0.0661
Epoch 393/400
1320/1320 [==============================

1320/1320 [==============================] - 3s 3ms/step - loss: 0.0677 - acc: 0.0664 - val_loss: 0.0930 - val_acc: 0.0667
Epoch 397/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.0676 - acc: 0.0664 - val_loss: 0.0937 - val_acc: 0.0665
Epoch 398/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.0666 - acc: 0.0665 - val_loss: 0.1001 - val_acc: 0.0629
Epoch 399/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.0696 - acc: 0.0657 - val_loss: 0.0916 - val_acc: 0.0661
Epoch 400/400
1320/1320 [==============================] - 3s 3ms/step - loss: 0.0660 - acc: 0.0667 - val_loss: 0.0938 - val_acc: 0.0657


In [0]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()
encoder_model.save("drive/chatbot/encoder_model.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_inputs (InputLayer)  (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 50)          87150     
_________________________________________________________________
lstm_1 (LSTM)                [(None, 50), (None, 50),  20200     
Total params: 107,350
Trainable params: 107,350
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.save("drive/chatbot/bot_model.h5")

/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:2379: UserWarning: Layer lstm_6 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 50) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


#### Create sampling model

In [0]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())
decoder_model.summary()
decoder_model.save('drive/chatbot/decoder_model.h5')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_inputs (InputLayer)     (None, None)         0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 50)     55550       decoder_inputs[0][0]             
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
lstm_6 (LS

/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:2379: UserWarning: Layer lstm_6 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'input_3:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'input_4:0' shape=(?, 50) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


#### Function to generate sequences

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [0]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

      

In [0]:
from keras import backend as K
import tensorflow as tf

# Create, compile and train model...

frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, "/drive/chatbot", "my_model.pb", as_text=False)

INFO:tensorflow:Froze 48 variables.
INFO:tensorflow:Converted 48 variables to const ops.


'/drive/chatbot/my_model.pb'

In [0]:
!ls drive/chatbot

checkpoint  keras_model.ckpt.data-00000-of-00001  keras_model.ckpt.meta
hin.txt     keras_model.ckpt.index


#### Look at the some translations

In [0]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', lines.eng[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: 0    shanky happy th bday shanky  🎂🎂🎂🎂🍫🍫🍫🍫🍫 httpsww...
Name: eng, dtype: object
Decoded sentence:  dandiya dandiya khi cute user user user user user user
-
Input sentence: 1    bad me 
Name: eng, dtype: object
Decoded sentence:  dandiya dandiya khi cute user user user user user user
-
Input sentence: 2    hi chachu phn kar rhe h na 
Name: eng, dtype: object
Decoded sentence:  dandiya dandiya khi cute user user user user user user
-
Input sentence: 3    ok 
Name: eng, dtype: object
Decoded sentence:  dandiya dandiya khi cute user user user user user user
-
Input sentence: 4    ok httpswwwairtelinsselfcarenormallogin 
Name: eng, dtype: object
Decoded sentence:  dandiya dandiya khi cute user user user user user user
-
Input sentence: 5    ha 
Name: eng, dtype: object
Decoded sentence:  dandiya dandiya khi cute user user user user user user
-
Input sentence: 6    aja ladeshwar tere liye aarti ki thali tayar r...
Name: eng, dtype: object
Decoded sentence:  dandiya dandiya 

-
Input sentence: 60    mere airtel number pe call kar 
Name: eng, dtype: object
Decoded sentence:  dandiya dandiya khi cute user user user user user user
-
Input sentence: 61    wah yar teko pata bhi nhi internet hamesha cha...
Name: eng, dtype: object
Decoded sentence:  dandiya dandiya khi cute user user user user user user
-
Input sentence: 62    ab kar ke dekh ho gaya na 
Name: eng, dtype: object
Decoded sentence:  dandiya dandiya khi cute user user user user user user
-
Input sentence: 63    rechargeckiya h kyunki gawar bye tc 
Name: eng, dtype: object
Decoded sentence:  dandiya dandiya khi cute user user user user user user
-
Input sentence: 64    i was the first one to do it 🤣 at least in my ...
Name: eng, dtype: object
Decoded sentence:  dandiya dandiya khi cute user user user user user user
-
Input sentence: 65    kya potty time pass ke liye ok h 
Name: eng, dtype: object
Decoded sentence:  dandiya dandiya khi cute user user user user user user
-
Input sentence: 66    logo ne 

In [0]:
def InputLine(line):
  input_line_data = np.zeros(
      (1, max_encoder_seq_length),
      dtype='float32')
  for t, word in enumerate(line.split()):
        input_line_data[0, t] = input_token_index[word]
  decoded_sentence=decode_sequence(input_line_data)    
  print(decoded_sentence)

In [0]:
InputLine("kya kre btao")

KeyError: ignored

In [0]:
!pip install bazel

  Could not find a version that satisfies the requirement bazel (from versions: )
No matching distribution found for bazel
